# Calculo de Probabilidade de Viagem

Este notebook calcula a probabilidade de uma pessoa infectada viajar para um determinadas cidade a qual possui conexão.

In [2]:
import pandas as pd
import numpy as np

In [4]:
df_municipio = pd.read_csv("../data/integrado/municipio.csv")
df_fluxo_aereo = pd.read_csv("../data/integrado/fluxo_aereo.csv")
df_fluxo_rodoviario = pd.read_csv("../data/integrado/fluxo_rodoviario.csv")

Funções de extração dos dados

In [104]:
def info_cidade(codigo_municipio, df_municipio, df_fluxo_aereo, df_fluxo_rodoviario):
    #Consultando valores nos dataframe a partir do código da cidade
    # total_populacao = df_municipio[df_municipio.cod_mun == codigo_municipio].populacao_2021 # 
    
    df_aereo = df_fluxo_aereo[df_fluxo_aereo.cod_mun_origem == codigo_municipio][['cod_mun_destino', 'total_passagens_2020']].rename(columns={"cod_mun_destino": "cod_mun"})
    
    
    df_rod_1 = df_fluxo_rodoviario[(df_fluxo_rodoviario.cod_mun_a == codigo_municipio)][['cod_mun_b', 'freq_saida_2016']].rename(columns={"cod_mun_b": "cod_mun"})
    df_rod_2 = df_fluxo_rodoviario[(df_fluxo_rodoviario.cod_mun_b == codigo_municipio)][['cod_mun_a', 'freq_saida_2016']].rename(columns={"cod_mun_a": "cod_mun"})
    df_rod = pd.concat([df_rod_1,df_rod_2], ignore_index=True)
    
    df_fluxo = df_aereo.merge(df_rod[['cod_mun', 'freq_saida_2016']], how='outer', left_on='cod_mun', right_on='cod_mun')
    # df_fluxo = df_aereo #TODO: Remover esta linha quando adicionar o rodoviario -> Bug do código 0
    df_fluxo.fillna(0, inplace = True)

    # df_fluxo = df_fluxo.merge(df_municipio[['cod_mun', 'populacao_2021']], how='left', left_on='cod_mun', right_on='cod_mun')
    
    # lista_fluxo = list(df_fluxo.itertuples(index=False, name=None))
    subset = df_fluxo[['cod_mun', 'total_passagens_2020', 'freq_saida_2016']]
    lista_fluxo = [tuple(x) for x in subset.to_numpy()]
    
    return lista_fluxo  ##--> Atualmente devolve (cod_dest, fluxo_aereo, fluxo_rod_veiculos)

In [105]:
info_cidade(2611606, df_municipio, df_fluxo_aereo, df_fluxo_rodoviario)

[(3550308.0, 287035.0, 38.0),
 (2304400.0, 71275.0, 91.0),
 (3304557.0, 97299.0, 32.0),
 (5300108.0, 88821.0, 26.0),
 (2927408.0, 41944.0, 38.0),
 (3106200.0, 22107.0, 14.0),
 (4314902.0, 30038.0, 0.0),
 (2611101.0, 12746.0, 63.0),
 (3509502.0, 21587.0, 19.0),
 (2605459.0, 20384.0, 0.0),
 (1302603.0, 31506.0, 0.0),
 (2211001.0, 11989.0, 66.0),
 (1501402.0, 19551.0, 19.0),
 (4106902.0, 20109.0, 7.0),
 (3205309.0, 12621.0, 6.0),
 (5208707.0, 18550.0, 43.0),
 (2111300.0, 12485.0, 21.0),
 (5103403.0, 17618.0, 16.0),
 (2800308.0, 6550.0, 49.0),
 (4108304.0, 5666.0, 2.0),
 (4208203.0, 10841.0, 0.0),
 (4205407.0, 11627.0, 0.0),
 (2307304.0, 4538.0, 28.0),
 (5002704.0, 9983.0, 7.0),
 (1100205.0, 6497.0, 1.0),
 (3543402.0, 0.0, 16.0),
 (4113700.0, 0.0, 0.0),
 (3549805.0, 0.0, 16.0),
 (3170206.0, 5779.0, 27.0),
 (2704302.0, 1529.0, 136.0),
 (2408102.0, 2217.0, 72.0),
 (1721000.0, 3557.0, 14.0),
 (1600303.0, 2582.0, 0.0),
 (2105302.0, 0.0, 6.0),
 (4209102.0, 0.0, 0.0),
 (4305108.0, 0.0, 0.0),
 (4

Calculo da Probabilidade

In [106]:
def calcular_probabilidade(cod_cidade_origem):
    FATOR_RODOV = 20 *365

    ## Calculando as probabilidade de viajar e estar infectado
    # total_populacao, lista_fluxo= info_cidade(cod_cidade_origem, df_municipio, df_fluxo_aereo, df_fluxo_rodoviario)
    lista_fluxo= info_cidade(cod_cidade_origem, df_municipio, df_fluxo_aereo, df_fluxo_rodoviario)
    fluxo_total = sum(j for i, j, k in lista_fluxo)
    fluxo_total += sum(k for i, j, k in lista_fluxo) * FATOR_RODOV ##Valor que tem que mudar, essa é minha estimativa


    prob_final = []
    if(fluxo_total!=0):
        for codigo_destino, fluxo_aereo, fluxo_rodo in lista_fluxo:
            # print(f"{codigo} - {fluxo_aereo} - {fluxo_rodo}")

            fluxo = fluxo_aereo + fluxo_rodo * FATOR_RODOV
            prob_viajar = fluxo / fluxo_total

            prob_final.append((cod_cidade_origem,int(codigo_destino), prob_viajar))
            # prob_final.append((codigo_destino, prob_viajar))

    return prob_final

In [7]:
df = pd.read_csv("../data/calculado/fluxo_prob.csv")
df_salvador = df[df.source == 2927408]
list_df = list(df_salvador.itertuples(index=False))

list_df.sort(key=lambda x:-x[2])
for cod_mun_origem, cod_mun_destino, prob in list_df:
    nome = df_municipio.loc[df_municipio.cod_mun==cod_mun_destino, 'nome_mun'].values[0]
    print(f"{nome} - {prob}")



Feira de Santana - 0.0329422868611331
Santa Bárbara - 0.0302729302830671
Cruz das Almas - 0.027603573705001
Conceição do Coité - 0.0267845665730944
Governador Mangabeira - 0.0211425174421821
Alagoinhas - 0.0192618343985447
Sapeaçu - 0.0182911592792479
Catu - 0.0178361553170776
Serrinha - 0.0164711434305665
Cachoeira - 0.0153488003238797
São Sebastião do Passé - 0.0148331291667533
Santo Amaro - 0.0145904603869291
Entre Rios - 0.0135591180726763
Pojuca - 0.0129221125256378
Santo Antônio de Jesus - 0.0125277757584235
Conceição da Feira - 0.0124367749659894
Milagres - 0.0119817710038191
Conceição do Almeida - 0.011921103808863
Capim Grosso - 0.0115267670416487
Teofilândia - 0.0114357662492147
Jequié - 0.0114054326517367
Tanquinho - 0.0113144318593026
Esplanada - 0.0113144318593026
Araças - 0.0107684271046982
Araci - 0.0107684271046982
Amélia Rodrigues - 0.0104650911299179
Conceição do Jacuípe - 0.009221413633319
São Félix - 0.0091000792434069
São Felipe - 0.0090697456459289
São Gonçalo dos

### Calcular a probabilidade de um municipio pra o outro


In [103]:
#Pegar a lista de municios
lista_mun = df_municipio["cod_mun"].tolist()

probabilidade_exportar = []

In [107]:
#Calcular todas as probabilidades de importação
for cod_mun in lista_mun:
    list_probabilidades = calcular_probabilidade(cod_mun)
    
    probabilidade_exportar.extend(list_probabilidades)
    

In [111]:
#Criando dataframe para salvar dados
df_probabilidades = pd.DataFrame(probabilidade_exportar, columns =['cod_origem', 'cod_destino', 'probabilidade'])

#Salvando dataframe para ser carregado no neo4j
df_probabilidades.to_csv("../data/calculado/fluxo_prob.csv", index=False)

Testando Grafos em Python

In [8]:
df = pd.read_csv("../data/calculado/fluxo_prob.csv")
df.head()

,source,target,weight
0,5200050,5216403,0.750000
1,5200050,5207352,0.250000
2,3100104,3156403,0.126943
3,3100104,3124807,0.036269
4,3100104,3170206,0.163212


In [9]:
df["inv_weight"] = 1 - df["weight"]

In [10]:
import networkx as nx

Graphtype = nx.DiGraph()
G = nx.from_pandas_edgelist(df, edge_attr='inv_weight', create_using=Graphtype)



In [11]:
# closeness_centrality(G, u=None, distance=None, wf_improved=True)a
dic_closeness  = nx.closeness_centrality(G, distance="inv_weight", wf_improved=True)

KeyboardInterrupt: 

In [29]:
[(k, v, df_municipio.loc[df_municipio.cod_mun==k, 'nome_mun'].values[0]) for k, v in sorted(dic_closeness.items(), key=lambda item: item[1], reverse=True)]

[(3550308, 0.6686728911733462, 'São Paulo'),
 (5300108, 0.5775525257039477, 'Brasília'),
 (3509502, 0.5773117405182246, 'Campinas'),
 (3106200, 0.5731344869293246, 'Belo Horizonte'),
 (5208707, 0.5721354265416834, 'Goiânia'),
 (3304557, 0.5663159047178048, 'Rio de Janeiro'),
 (4106902, 0.554951862797327, 'Curitiba'),
 (3543402, 0.5522297052216999, 'Ribeirão Preto'),
 (4104808, 0.5517943883751333, 'Cascavel'),
 (4314902, 0.5493373761936324, 'Porto Alegre'),
 (3170206, 0.548622815301834, 'Uberlândia'),
 (2304400, 0.5480930417899252, 'Fortaleza'),
 (4115200, 0.5477957609440036, 'Maringá'),
 (2927408, 0.5466684354105059, 'Salvador'),
 (3549805, 0.5453563992904542, 'São José do Rio Preto'),
 (5002704, 0.5424083835448464, 'Campo Grande'),
 (2611606, 0.542211443444339, 'Recife'),
 (5103403, 0.5415032969993407, 'Cuiabá'),
 (2903201, 0.5409488834814986, 'Barreiras'),
 (4205407, 0.5408773352755915, 'Florianópolis'),
 (2211001, 0.539541340569399, 'Teresina'),
 (2933307, 0.5394256416225349, 'Vitór

In [26]:
dic_closeness_rev  = nx.closeness_centrality(G.reverse(), distance="inv_weight", wf_improved=True)

In [27]:
[(k, v, df_municipio.loc[df_municipio.cod_mun==k, 'nome_mun'].values[0]) for k, v in sorted(dic_closeness_rev.items(), key=lambda item: item[1], reverse=True)]

[(3135456, 0.5795000774696468, 'Jenipapo de Minas'),
 (3556503, 0.579500032064931, 'Várzea Paulista'),
 (3519071, 0.5795000166273291, 'Hortolândia'),
 (3549953, 0.5794999330826749, 'São Lourenço da Serra'),
 (3550308, 0.5794380816029032, 'São Paulo'),
 (3509205, 0.5303589907739246, 'Cajamar'),
 (3509502, 0.5122983024123088, 'Campinas'),
 (3170008, 0.5103287929670454, 'Ubaí'),
 (5300108, 0.510280716776476, 'Brasília'),
 (3136603, 0.5085424678365684, 'Nova União'),
 (3134608, 0.5085422211290617, 'Jaboticatubas'),
 (3106200, 0.5084947912296353, 'Belo Horizonte'),
 (5201801, 0.5064039017862701, 'Aragoiânia'),
 (5208707, 0.5063570485967219, 'Goiânia'),
 (3304557, 0.504081180359178, 'Rio de Janeiro'),
 (4103107, 0.49600967588061634, 'Bocaiúva do Sul'),
 (4102307, 0.4960096380286133, 'Balsa Nova'),
 (4106902, 0.4959642803637431, 'Curitiba'),
 (4104808, 0.4928873223315096, 'Cascavel'),
 (1718303, 0.49283786627440346, 'Praia Norte'),
 (3501707, 0.4928378408636088, 'Américo Brasiliense'),
 (3543